In [8]:
import pandas as pd
from pyspark.sql.functions import (col, rand, udf, collect_set, collect_list, concat, array_distinct, flatten, when, length)
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType, FloatType
from unidecode import unidecode
import re
from functools import reduce
import numpy as np
import tensorflow as tf
import keras
from fuzzywuzzy import fuzz

from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Bidirectional, GlobalMaxPooling1D, Layer, Masking, Lambda, Permute
import os
import pickle
from keras.layers import Bidirectional
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import functools
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout, concatenate
from keras.activations import relu
from sklearn.model_selection import train_test_split

def custom_standardization(input_data):
    lowercase     = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    stripped_punc = tf.strings.regex_replace(stripped_html, 
                             "[%s]" % re.escape(string.punctuation), "")
    return stripped_punc
    
def char_split(input_data):
  return tf.strings.unicode_split(input_data, 'UTF-8')


def get_conv_pool(x_input, max_len, suffix, n_grams=[2,3,5,8, 13], feature_maps=128):
    branches = []
    for n in n_grams:
        branch = tf.keras.layers.Conv1D(filters=feature_maps, kernel_size=n, activation=relu,
                        name='Conv_' + suffix + '_' + str(n))(x_input)
        branch = tf.keras.layers.MaxPooling1D(pool_size=max_len - n + 1,
                                              strides=1, padding='valid',
                              name='MaxPooling_' + suffix + '_' + str(n))(branch)
        branch = tf.keras.layers.Flatten(name='Flatten_' + suffix + '_' + str(n))(branch)
        branches.append(branch)
    return branches

class poiNames_C2V:
    
    def __init__(self, embedding_dim:int, char_to_index:dict):

        self.char_to_index = char_to_index
        self.index_to_char = {char_to_index[char]: char for char in char_to_index}
        self.input_size = MAX_NAME_LEN
        self.embedding_dim = embedding_dim
        self.vocabulary_size = len(char_to_index)

        input_sequence = tf.keras.layers.Input(shape=self.input_size, name='input_sequence')
        # x = tf.keras.layers.Masking(mask_value=0)(input_sequence)
        x = tf.keras.layers.Embedding(input_dim=self.input_size, output_dim=self.embedding_dim, mask_zero=True)(input_sequence)
        self.embedding_layer = tf.keras.models.Model(inputs=[input_sequence], outputs=x)

        sequences_emb = self.embedding_layer(input_sequence)
        branches = get_conv_pool(sequences_emb, max_len=100, suffix='unique')
        z = concatenate(branches, axis=-1)
        self.convolutions_layer = tf.keras.models.Model(inputs=[input_sequence], outputs=z)

        left_branch_input = tf.keras.layers.Input(shape=(self.input_size, 1), name='left_branch_input')
        right_branch_input = tf.keras.layers.Input(shape=(self.input_size, 1),name='right_branch_input')

        left_branch_features = self.convolutions_layer(left_branch_input)
        right_branch_features = self.convolutions_layer(right_branch_input)

        
        
        product_layer = tf.keras.layers.Multiply()([left_branch_features, right_branch_features])
        difference_layer = tf.keras.layers.Subtract()([left_branch_features, right_branch_features])
        concat_layer = tf.keras.layers.Concatenate(axis=1)([left_branch_features, right_branch_features])
        
        representation_layer = tf.keras.layers.Concatenate(axis=1)([
                                                                    concat_layer, product_layer, difference_layer, 
                                                                    ])
        x = tf.keras.layers.Dense(1024, activation='relu')(representation_layer)
        x = tf.keras.layers.Dropout(0.4)(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.4)(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        model_output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        
        self.model = tf.keras.models.Model(inputs=[left_branch_input, right_branch_input], outputs=model_output)
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',
                                                              tf.keras.metrics.Precision(),
                                                              tf.keras.metrics.Recall()])
        
    def fit(
        self, name_pairs, target, max_epochs:int, patience:int, validation_split:float, batch_size:int, callbacks: iter):
        
        X1 = encode_sparsed_list(name_pairs[:, 0],char_to_index_dict=self.char_to_index, max_len=MAX_NAME_LEN)
        X2 = encode_sparsed_list(name_pairs[:, 1], char_to_index_dict=self.char_to_index, max_len=MAX_NAME_LEN)
        X1 = np.expand_dims(X1, axis=-1)
        X2 = np.expand_dims(X2, axis=-1)
        X1_pad = tf.keras.preprocessing.sequence.pad_sequences(X1)
        X2_pad = tf.keras.preprocessing.sequence.pad_sequences(X2)
        
        self.model.fit((X1_pad, X2_pad), target,verbose=1,
                    batch_size=batch_size, epochs=max_epochs,
                    validation_split=validation_split,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)]
                            + callbacks)


    def save_model(self, path_to_model):
        '''
        Saves trained model to directory.
    
        :param path_to_model: str, path to save model.
        '''
    
        if not os.path.exists(path_to_model):
            os.makedirs(path_to_model)
        
        self.model.save_weights(path_to_model + '/weights.h5')
    
        with open(path_to_model + '/model.pkl', 'wb') as f:
            pickle.dump([self.embedding_dim, self.char_to_index], f, protocol=2)


    def load_model(self, path):
        '''
        Loads trained model.
    
        :param path: loads model from `path`.
    
        :return c2v_model: Chars2Vec object, trained model.
        '''
        path_to_model = path
    
        with open(path_to_model + '/model.pkl', 'rb') as f:
            structure = pickle.load(f)
            embedding_dim, char_to_index = structure[0], structure[1]
    
        model = poiNames_C2V(embedding_dim=embedding_dim, char_to_index=char_to_index)
        model.model.load_weights(path_to_model + '/weights.h5')
        model.model.compile(optimizer='adam', loss='mae')
    
        return model


    def vectorize(self, iterable: iter) -> np.array:
        
        X = encode_sparsed_list(iterable,char_to_index_dict=self.char_to_index)
        X = np.expand_dims(X, axis=-1)
        X_pad = tf.keras.preprocessing.sequence.pad_sequences(X)

        return X_pad

In [9]:
MAX_NAME_LEN = 100
char_to_index = catalog.load('vocabulary_file')

[09/03/23 21:13:36] INFO     Loading data from 'vocabulary_file' (JSONDataSet)...               ]8;id=123499;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=993807;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

In [10]:
poi_c2v = poiNames_C2V(150, char_to_index)

2023-09-03 21:13:36.723372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
X_train = catalog.load('df_training_data')
X_train[X_train['target'] == 0]

                    INFO     Loading data from 'df_training_data' (CSVDataSet)...               ]8;id=892652;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=484033;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

,reference_id,brand,name,alt_name,tags,latitude,longitude,category_fields,name_normalized,alt_name_normalized,run_id,country,target
29824,47905637612,NaN,Olabarrieta,Casa Pancho,"""license""=>""ODbL"", ""osm_uid""=>""1"", ""alt_name""=...",43.273544,-3.045194,['place=locality'],olabarrieta,casa pancho,20230901-test-0006,esp,0
29825,33939964186,NaN,Hotel Selu,Miranda,"""fax""=>""+(34)-(957)-478376"", ""user""=>""tomtom"",...",37.884247,-4.783952,['tourism=hotel'],hotel selu,miranda,20230901-test-0006,esp,0
29826,33920295151,NaN,Centro Privado de Educación Infantil Jugar Y S...,Enrique Tomás Jamon-Tapas-Vino,"""user""=>""tomtom"", ""phone""=>""+34600257838"", ""li...",40.588801,-3.708725,['amenity=kindergarten'],centro privado de educacion infantil jugar y s...,enrique tomas jamon tapas vino,20230901-test-0006,esp,0
29827,9151154135,NaN,Mi Peluquería,Hawaii,"""phone""=>""+34 942 31 25 75"", ""license""=>""ODbL""...",43.469193,-3.808571,['shop=hairdresser'],mi peluqueria,hawaii,20230901-test-0006,esp,0
29828,34029676494,Domino's,Domino's,Pizza Tutto,"""user""=>""tomtom"", ""phone""=>""+34910900924"", ""cu...",40.402120,-3.678620,['amenity=restaurant'],dominos,pizza tutto,20230901-test-0006,esp,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59643,33933382525,NaN,Llesca,Pico Las Rebequeras,"""user""=>""tomtom"", ""phone""=>""+34925541064"", ""cu...",40.121636,-3.849563,['amenity=restaurant'],llesca,pico las rebequeras,20230901-test-0006,esp,0
59644,33950966775,NaN,Centro Privado Autorizado de Enseñanzas Deport...,Restaurante Bodegon Del Mar,"""user""=>""tomtom"", ""phone""=>""+34912115601"", ""sc...",40.373642,-3.920745,['amenity=school'],centro privado autorizado de ensenanzas deport...,restaurante bodegon del mar,20230901-test-0006,esp,0
59645,34005477933,NaN,Restaurante Rincon De Pedro,Apartamento Gran Via Callao,"""user""=>""tomtom"", ""email""=>""rincondepedro@live...",38.077685,-0.649647,['amenity=restaurant'],restaurante rincon de pedro,apartamento gran via callao,20230901-test-0006,esp,0
59646,2904773690,NaN,Font de l'Aueta,Moka,"""license""=>""ODbL"", ""osm_uid""=>""1"", ""alt_name""=...",38.736821,-0.353900,['natural=spring'],font de laueta,moka,20230901-test-0006,esp,0


,reference_id,brand,name,alt_name,tags,latitude,longitude,category_fields,name_normalized,alt_name_normalized,run_id,country,target
29824,47905637612,NaN,Olabarrieta,Las Barrietas,"""license""=>""ODbL"", ""osm_uid""=>""1"", ""alt_name""=...",43.273544,-3.045194,['place=locality'],olabarrieta,las barrietas,20230901-test-0006,esp,0
29825,33939964186,NaN,Hotel Selu,Sercotel Hotels Selu Hotel,"""fax""=>""+(34)-(957)-478376"", ""user""=>""tomtom"",...",37.884247,-4.783952,['tourism=hotel'],hotel selu,sercotel hotels selu hotel,20230901-test-0006,esp,0
29826,33920295151,NaN,Centro Privado de Educación Infantil Jugar Y S...,Jugar Y Sentir,"""user""=>""tomtom"", ""phone""=>""+34600257838"", ""li...",40.588801,-3.708725,['amenity=kindergarten'],centro privado de educacion infantil jugar y s...,jugar y sentir,20230901-test-0006,esp,0
29827,9151154135,NaN,Mi Peluquería,Peluquería Nuria Bailo,"""phone""=>""+34 942 31 25 75"", ""license""=>""ODbL""...",43.469193,-3.808571,['shop=hairdresser'],mi peluqueria,peluqueria nuria bailo,20230901-test-0006,esp,0
29828,34029676494,Domino's,Domino's,Domino's Tellez,"""user""=>""tomtom"", ""phone""=>""+34910900924"", ""cu...",40.402120,-3.678620,['amenity=restaurant'],dominos,dominos tellez,20230901-test-0006,esp,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59643,33933382525,NaN,Llesca,Gastrotaberna Illescas,"""user""=>""tomtom"", ""phone""=>""+34925541064"", ""cu...",40.121636,-3.849563,['amenity=restaurant'],llesca,gastrotaberna illescas,20230901-test-0006,esp,0
59644,33950966775,NaN,Centro Privado Autorizado de Enseñanzas Deport...,Ufedema-Centro de Enseñanzas deportivas,"""user""=>""tomtom"", ""phone""=>""+34912115601"", ""sc...",40.373642,-3.920745,['amenity=school'],centro privado autorizado de ensenanzas deport...,ufedema centro de ensenanzas deportivas,20230901-test-0006,esp,0
59645,34005477933,NaN,Restaurante Rincon De Pedro,Rincon De Pedro,"""user""=>""tomtom"", ""email""=>""rincondepedro@live...",38.077685,-0.649647,['amenity=restaurant'],restaurante rincon de pedro,rincon de pedro,20230901-test-0006,esp,0
59646,2904773690,NaN,Font de l'Aueta,la Font de Cossi,"""license""=>""ODbL"", ""osm_uid""=>""1"", ""alt_name""=...",38.736821,-0.353900,['natural=spring'],font de laueta,la font de cossi,20230901-test-0006,esp,0


In [15]:
catalog.load('df_negative_pairs').show()

[09/03/23 21:20:20] INFO     Loading data from 'df_negative_pairs' (SparkDataSet)...            ]8;id=597276;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=767231;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

+------------+--------+--------------------+--------------------+--------------------+----------+----------+--------------------+--------------------+--------------------+------------------+-------+
|reference_id|   brand|                name|            alt_name|                tags|  latitude| longitude|     category_fields|     name_normalized| alt_name_normalized|            run_id|country|
+------------+--------+--------------------+--------------------+--------------------+----------+----------+--------------------+--------------------+--------------------+------------------+-------+
| 46320957956|    null|Cámaras de tráfic...|Escultor Viciano;...|"license"=>"ODbL"...|39.9844658|-0.0389732|[man_made=surveil...|camaras de trafic...|escultor viciano ...|20230901-test-0005|    esp|
| 33890660161|    null|       Bar El Carmen|           El Carmen|"user"=>"tomtom",...| 39.406133| -2.424481|[amenity=restaurant]|       bar el carmen|           el carmen|20230901-test-0005|    esp|
| 340

In [5]:
X_val = catalog.load('validation')

[09/03/23 20:47:29] INFO     Loading data from 'validation' (CSVDataSet)...                     ]8;id=809163;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=689885;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

In [6]:
catalog.list()


[
    'df_normalized_pairs',
    'df_negative_pairs',
    'query_schema',
    'df_schema',
    'spatial_query',
    'df_response',
    'df_raw_names_sample',
    'vocabulary_file',
    'train',
    'test',
    'validation',
    'tensorflow_model',
    'training_history_loss',
    'confussion_matrix_model',
    'confussion_matrix_validation',
    'classification_logs',
    'parameters',
    'params:run_id',
    'params:country',
    'params:provider',
    'params:sample_round',
    'params:query_parameters',
    'params:query_parameters.om',
    'params:query_parameters.om.schema_query',
    'params:query_parameters.om.data_query',
    'params:query_parameters.tt',
    'params:query_parameters.tt.schema_query',
    'params:query_parameters.tt.data_query',
    'params:query_parameters.osm',
    'params:query_parameters.osm.schema_query',
    'params:query_parameters.osm.data_query',
    'params:has_custom_credentials',
    'params:custom_credentials',
    'params:custom_credentials.host

In [10]:
char_to_index = catalog.load('vocabulary_file')

[09/03/23 20:50:11] INFO     Loading data from 'vocabulary_file' (JSONDataSet)...               ]8;id=941861;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=979594;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

In [11]:
X1_train = X_train['name_normalized'].values
X2_train = X_train['alt_name_normalized'].values
target_train = X_train['target'].values

In [12]:
X1_val = X_val['name_normalized'].values
X2_val = X_val['alt_name_normalized'].values
target_val = X_val['target'].values

In [15]:
init_parameters = catalog.load('params:model_parameters.init')

model = char2vecCNN(
    input_size=init_parameters['input_size'],
    embedding_dim = init_parameters['input_size'],
    char_to_index = char_to_index)

fit_parameters = catalog.load('params:model_parameters.fit')
history = History()

model.fit(
    training_pairs=(X1_train, X2_train),
    target=target_train,
    max_epochs=fit_parameters['max_epochs'],
    patience=fit_parameters['patience'],
    validation_pairs=((X1_val, X2_val), (target_val)),
    batch_size=fit_parameters['batch_size'],
    callbacks=[history]
)

[09/03/23 20:50:35] INFO     Loading data from 'params:model_parameters.init'                   ]8;id=604040;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=125416;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from 'params:model_parameters.fit' (MemoryDataset)... ]8;id=237628;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=887535;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

(array(['farmacia calle huertas', 'cal deu', 'restaurante chino hongkong',
       ..., 'autoescuela ronda', 'vincci seleccion la plantacion del sur',
       'goikogane'], dtype=object), array(['farmacia huertas', 'el molinot', 'hong kong', ...,
       'autoescuela nueva ronda', 'vincci la plantacion', 'atxandita'],
      dtype=object))
Epoch 1/5
124/187 [==================>...........] - ETA: 17s - loss: 0.6942 - accuracy: 0.4968 - precision_1: 0.4981 - recall_1: 0.5680

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 11>:11                                                                            │
│                                                                                                  │
│    8 fit_parameters = catalog.load('params:model_parameters.fit')                                │
│    9 history = History()                                                                         │
│   10                                                                                             │
│ ❱ 11 model.fit(                                                                                  │
│   12 │   training_pairs=(X1_train, X2_train),                                                    │
│   13 │   target=target_train,                                                                    │
│   14 │   max_epochs=fit_parameters['max_epochs'],                                                │
│                                                                                                  │
│ /workspace/src/entity_name_matching_dnn/pipelines/train_model/model.py:130 in fit                │
│                                                                                                  │
│   127 │   │                                                                                      │
│   128 │   │   _callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=pati   │
│   129 │   │                                                                                      │
│ ❱ 130 │   │   self.model.fit(                                                                    │
│   131 │   │   │   (X1, X2),                                                                      │
│   132 │   │   │   target,                                                                        │
│   133 │   │   │   verbose=1,                                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/keras/utils/traceback_utils.py:65 in error_handler        │
│                                                                                                  │
│    62 │   │                                                                                      │
│    63 │   │   filtered_tb = None                                                                 │
│    64 │   │   try:                                                                               │
│ ❱  65 │   │   │   return fn(*args, **kwargs)                                                     │
│    66 │   │   except Exception as e:                                                             │
│    67 │   │   │   filtered_tb = _process_traceback_frames(e.__traceback__)                       │
│    68 │   │   │   # To get the full stack trace, call:                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/keras/engine/training.py:1650 in fit                      │
│                                                                                                  │
│   1647 │   │   │   │   │   │   │   _r=1,                                                         │
│   1648 │   │   │   │   │   │   ):                                                                │
│   1649 │   │   │   │   │   │   │   callbacks.on_train_batch_begin(step)                          │
│ ❱ 1650 │   │   │   │   │   │   │   tmp_logs = self.train_function(iterator)                      │
│   1651 │   │   │   │   │   │   │   if data_handler.should_sync:                                  │
│   1652 │   │   │   │   │   │   │   │   context.async_wait()                                      │
│   1653 │   │   │   │   │   │   │   # No error, now safe to 

In [17]:
X_train = catalog.load('train')

[09/03/23 21:38:20] INFO     Loading data from 'train' (CSVDataSet)...                          ]8;id=828413;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=730002;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

In [20]:
X_train.loc[[1,4,2]]

,reference_id,brand,name,alt_name,tags,latitude,longitude,category_fields,name_normalized,alt_name_normalized,run_id,country,target
1,46277096854,NaN,Cal Déu,el Molinot,"""license""=>""ODbL"", ""osm_uid""=>""1"", ""alt_name""=...",42.080492,1.781784,['place=isolated_dwelling'],cal deu,el molinot,20230901-test-0005,esp,0
4,33913358616,NaN,El Triunfo,Hostal El Triunfo,"""user""=>""tomtom"", ""email""=>""reservas@hostaltri...",37.878365,-4.778780,['tourism=hotel'],el triunfo,hostal el triunfo,20230901-test-0005,esp,0
2,34007673148,NaN,Restaurante Chino Hongkong,Hong-Kong,"""user""=>""tomtom"", ""phone""=>""+34922411308"", ""cu...",28.687215,-17.772395,['amenity=restaurant'],restaurante chino hongkong,hong kong,20230901-test-0005,esp,1


In [32]:
indexes = X_train.index.tolist()
np.random.shuffle(indexes)


array(['Boutique-hotel - Posada Terra Santa', 'Restaurante El Principe',
       'Cala de Gulpiyuri', ..., 'Jamoneria La Bellota', 'Gaztarri',
       'La Móra'], dtype=object)

In [35]:
X_train

,reference_id,brand,name,alt_name,tags,latitude,longitude,category_fields,name_normalized,alt_name_normalized,run_id,country,target,new_alt_names
0,33963143455,NaN,Farmacia Calle Huertas,Farmacia Huertas,"""user""=>""tomtom"", ""email""=>""elenaramiro@hotmai...",40.413902,-3.699700,['amenity=pharmacy'],farmacia calle huertas,farmacia huertas,20230901-test-0005,esp,0,Farmacia Huertas
1,46277096854,NaN,Cal Déu,el Molinot,"""license""=>""ODbL"", ""osm_uid""=>""1"", ""alt_name""=...",42.080492,1.781784,['place=isolated_dwelling'],cal deu,el molinot,20230901-test-0005,esp,0,el Molinot
2,34007673148,NaN,Restaurante Chino Hongkong,Hong-Kong,"""user""=>""tomtom"", ""phone""=>""+34922411308"", ""cu...",28.687215,-17.772395,['amenity=restaurant'],restaurante chino hongkong,hong kong,20230901-test-0005,esp,1,Hong-Kong
3,33956757671,Five Guys,Five Guys,Five Guys Bilbao Plaza Circular,"""user""=>""tomtom"", ""phone""=>""+34946662961"", ""li...",43.261337,-2.926707,['amenity=fast_food'],five guys,five guys bilbao plaza circular,20230901-test-0005,esp,0,Five Guys Bilbao Plaza Circular
4,33913358616,NaN,El Triunfo,Hostal El Triunfo,"""user""=>""tomtom"", ""email""=>""reservas@hostaltri...",37.878365,-4.778780,['tourism=hotel'],el triunfo,hostal el triunfo,20230901-test-0005,esp,0,Hostal El Triunfo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47713,33908137014,NaN,Taberna La Campa,La Campa,"""user""=>""tomtom"", ""phone""=>""+34985608069"", ""cu...",43.181736,-5.344619,['amenity=restaurant'],taberna la campa,la campa,20230901-test-0005,esp,0,La Campa
47714,33947006140,Domino's,Domino's,Domino's Velazquez,"""user""=>""tomtom"", ""phone""=>""+34952343104"", ""cu...",36.698340,-4.447830,['amenity=restaurant'],dominos,dominos velazquez,20230901-test-0005,esp,1,Domino's Velazquez
47715,33944581636,NaN,Autoescuela Ronda,Autoescuela Nueva Ronda,"""user""=>""tomtom"", ""email""=>""Gestion@aeronda.co...",40.309041,-3.447413,['amenity=driving_school'],autoescuela ronda,autoescuela nueva ronda,20230901-test-0005,esp,0,Autoescuela Nueva Ronda
47716,33997412510,NaN,Vincci Seleccion La Plantacion Del Sur,Vincci La Plantacion,"""fax""=>""+(34)-(922)-712750"", ""user""=>""tomtom"",...",28.094286,-16.742963,['tourism=hotel'],vincci seleccion la plantacion del sur,vincci la plantacion,20230901-test-0005,esp,1,Vincci La Plantacion


In [36]:
X_train[['alt_name', 'new_alt_names']]

,alt_name,new_alt_names
0,Farmacia Huertas,Farmacia Huertas
1,el Molinot,el Molinot
2,Hong-Kong,Hong-Kong
3,Five Guys Bilbao Plaza Circular,Five Guys Bilbao Plaza Circular
4,Hostal El Triunfo,Hostal El Triunfo
...,...,...
47713,La Campa,La Campa
47714,Domino's Velazquez,Domino's Velazquez
47715,Autoescuela Nueva Ronda,Autoescuela Nueva Ronda
47716,Vincci La Plantacion,Vincci La Plantacion


In [25]:
import pyspark
def generate_training_examples(df: pyspark.sql.DataFrame
                               ) -> pyspark.sql.DataFrame:
    
    #TODO: Implement more sophisticated negative examples

    positive_examples = df.toPandas()
    positive_examples['target'] = 1

    negative_examples = positive_examples.copy()
    negative_examples['target'] = 0

    np.random.seed(1)
    indexes = positive_examples.index.tolist()
    np.random.shuffle(indexes)

    negative_examples['alt_name'] = positive_examples.loc[indexes, 'alt_name']
    negative_examples['alt_name_normalized'] = positive_examples.loc[indexes, 'alt_name_normalized']

    training_data = pd.concat([positive_examples, negative_examples], axis=0)

    return training_data

In [26]:
normalized_pairs = catalog.load('df_normalized_pairs')

[09/03/23 22:07:09] INFO     Loading data from 'df_normalized_pairs' (SparkDataSet)...          ]8;id=832650;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=289496;file:///usr/local/lib/python3.9/dist-packages/kedro/io/data_catalog.py#475\475]8;;\

In [27]:
normalized_pairs.count()

29824

In [28]:
import numpy as np
import pandas as pd

In [29]:
train = generate_training_examples(normalized_pairs)

In [30]:
train

,reference_id,brand,name,alt_name,tags,latitude,longitude,category_fields,name_normalized,alt_name_normalized,run_id,country,target
0,47905637612,None,Olabarrieta,Las Barrietas,"""license""=>""ODbL"", ""osm_uid""=>""1"", ""alt_name""=...",43.273544,-3.045194,[place=locality],olabarrieta,las barrietas,20230901-test-0006,esp,1
1,33939964186,None,Hotel Selu,Sercotel Hotels Selu Hotel,"""fax""=>""+(34)-(957)-478376"", ""user""=>""tomtom"",...",37.884247,-4.783952,[tourism=hotel],hotel selu,sercotel hotels selu hotel,20230901-test-0006,esp,1
2,33920295151,None,Centro Privado de Educación Infantil Jugar Y S...,Jugar Y Sentir,"""user""=>""tomtom"", ""phone""=>""+34600257838"", ""li...",40.588801,-3.708725,[amenity=kindergarten],centro privado de educacion infantil jugar y s...,jugar y sentir,20230901-test-0006,esp,1
3,9151154135,None,Mi Peluquería,Peluquería Nuria Bailo,"""phone""=>""+34 942 31 25 75"", ""license""=>""ODbL""...",43.469193,-3.808571,[shop=hairdresser],mi peluqueria,peluqueria nuria bailo,20230901-test-0006,esp,1
4,34029676494,Domino's,Domino's,Domino's Tellez,"""user""=>""tomtom"", ""phone""=>""+34910900924"", ""cu...",40.402120,-3.678620,[amenity=restaurant],dominos,dominos tellez,20230901-test-0006,esp,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29819,33933382525,None,Llesca,Gastrotaberna Illescas,"""user""=>""tomtom"", ""phone""=>""+34925541064"", ""cu...",40.121636,-3.849563,[amenity=restaurant],llesca,gastrotaberna illescas,20230901-test-0006,esp,0
29820,33950966775,None,Centro Privado Autorizado de Enseñanzas Deport...,Ufedema-Centro de Enseñanzas deportivas,"""user""=>""tomtom"", ""phone""=>""+34912115601"", ""sc...",40.373642,-3.920745,[amenity=school],centro privado autorizado de ensenanzas deport...,ufedema centro de ensenanzas deportivas,20230901-test-0006,esp,0
29821,34005477933,None,Restaurante Rincon De Pedro,Rincon De Pedro,"""user""=>""tomtom"", ""email""=>""rincondepedro@live...",38.077685,-0.649647,[amenity=restaurant],restaurante rincon de pedro,rincon de pedro,20230901-test-0006,esp,0
29822,2904773690,None,Font de l'Aueta,la Font de Cossi,"""license""=>""ODbL"", ""osm_uid""=>""1"", ""alt_name""=...",38.736821,-0.353900,[natural=spring],font de laueta,la font de cossi,20230901-test-0006,esp,0
